In [40]:
import pandas as pd
import numpy as np

# Extracting Blosum62 Table 
- From .txt to dictionary as a reference 

In [41]:
def get_the_blosum_matrix(file):
    with open(file) as f:
        header = f.readline().upper().strip().replace('  ',' ')
        protein = header.split(' ')
        score = {}
        for line in f.readlines():
            row = line.upper().strip().replace('  ',' ').split(' ')
            protein2 = row[0]
            protein2_score = {}
            for i in range(len(protein)):
                protein2_score[protein[i]] = int(row[i+1])
            score[protein2] = protein2_score
        return score


# Alignment function
- traceback(matrix, row, col): To traceback the alignment and provide aligned seq in the first_seq and second_seq list
- alignment(x,y): To create a matrix with initialization column and row and to provide score values of each cell matrix based on the given rule 

In [44]:
first_seq = [] # seq assigned in y variable
second_seq = [] # seq assigned in x variable
def traceback(matrix, row, col):
    if col != 0:

        if row != 0 : # run function only if the matrix cell is not in the upper left matrix (zero index)
            
            rule_1 = matrix[row - 1][col - 1] + blosum_score[x[row-1]][y[col-1]]
            rule_2 = matrix[row][col - 1] - 1
            rule_3 = matrix[row - 1][col] - 1

            max_score = max(
                        rule_1,
                        rule_2,
                        rule_3)

            if max_score == rule_1: # the seq from the index x and y are aligned and will return the matching sequence
                first_seq.append(y[col-1])
                second_seq.append(x[row-1])
                return traceback(matrix, row - 1, col - 1) # traceback going to the above left cell
            
            elif max_score == rule_2: # no matching sequence, gap the nucleotide on the second seq
                first_seq.append(y[col-1])
                second_seq.append("-")
                return traceback(matrix, row, col - 1) # traceback going to the left cell

            elif max_score == rule_3: # no matching sequence, gap the nucleotide on the first seq
                first_seq.append("-")
                second_seq.append(x[row-1])
                return traceback(matrix, row - 1, col) # traceback going to the above cell

        else:
            first_seq.append(y[col-1])
            second_seq.append("-")

In [45]:
def alignment(x,y):

    col = len(y)
    row = len(x)
    matrix = [[0 for i in range(col + 1)] for j in range(row + 1)] # to provide basic matrix with the length of seq x and seq y

    for i in range(1, row + 1): # to provide initialization row for seq x
        matrix[i][0] = 0

    for j in range(1, col + 1): # to provide initialization row for seq y
        matrix[0][j] = 0

    for i in range(1, row + 1): 
        for j in range(1, col + 1):
            matrix[i][j] = max(
                matrix[i - 1][j - 1] + blosum_score[x[i-1]][y[j-1]] ,
                matrix[i - 1][j] - 1,
                matrix[i][j - 1] - 1,)
    traceback(matrix, row, col)
    x_index,y_index = list(x), list(y)
    x_index.insert(0,0)
    y_index.insert(0,0)
    df = pd.DataFrame(matrix, columns = y_index)
    df.index = x_index
    return df  

# Alignment sequences 
- Alignment sequences assigned in variable x and y respectively
- Variable y is referred as the first sequence
- Variable x is referred as the second sequence

In [47]:
blosum_score = get_the_blosum_matrix("blosum62.txt")
matrix = alignment(x,y)


In [75]:
y = 'MLKPVTK'
x = 'IEFIS'
print(f'Aligning sequence {y} as column index, and {x} as row index\n')
print(f'Alignment matrix:\n \n{matrix}\n') 
print('First sequence :')
print(*np.array(first_seq[::-1])) # reversed traceback sequence
print('Second sequence :')
print(*np.array(second_seq[::-1])) # reversed traceback sequence

Aligning sequence -MLKPVTK as column index, and -IEFIS as row index

Alignment matrix:
 
   0  M  L  K  P  V  T  K
0  0  0  0  0  0  0  0  0
I  0  1  2  1  0  3  2  1
E  0  0  1  3  2  2  2  3
F  0  0  0  2  1  1  1  2
I  0  1  2  1  0  4  3  2
S  0  0  1  2  1  3  5  4

First sequence :
M L K - P V T K
Second sequence :
- I E F - I S -


# Traceback Route
- Traceback route is highlighted in the yellow

In [74]:
matrix

,0,M,L,K,P,V,T,K
0,0,0,0,0,0,0,0,0
I,0,1,2,1,0,3,2,1
E,0,0,1,3,2,2,2,3
F,0,0,0,2,1,1,1,2
I,0,1,2,1,0,4,3,2
S,0,0,1,2,1,3,5,4
